In [68]:
# first neural network with keras tutorial
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from keras import backend as K
from sklearn.model_selection import train_test_split
import pandas as pd

In [69]:
df = pd.read_pickle('../env_specs/prediction_data.pkl')
df.drop(['ProbeFileID','IsTarget'], axis=1, inplace=True)
# replace nans with -1
df = df.fillna(-1)

In [70]:
X,Y = df.drop(['label'], axis=1).values, df['label'].values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [72]:
results = {}

for h1 in [25, 50, 100, 200, 400]:
    for h2 in [25, 50, 100, 200, 400]:
        for h3 in [0, 50]:
            def auc(y_true, y_pred):
                auc = tf.metrics.auc(y_true, y_pred)[1]
                K.get_session().run(tf.local_variables_initializer())
                return auc
            
            X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=h1*h2)

            # define the keras model
            model = Sequential()
            model.add(Dense(h1, input_dim=X.shape[1], activation='relu'))
            model.add(Dense(h2, input_dim=X.shape[1], activation='relu'))
            if h3 > 0:
                model.add(Dense(h3, input_dim=X.shape[1], activation='relu'))
            # model.add(Dense(50, input_dim=X.shape[1], activation='relu'))
            model.add(Dense(1, activation='sigmoid'))

            # compile the keras model
            model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', auc])

            # fit the keras model on the dataset
            model.fit(X_train, Y_train, epochs=20, batch_size=30)

            # evaluate the keras model
            _, accuracy, auc = model.evaluate(X_test, Y_test)
            results['{},{},{}'] = [accuracy, auc]
            print('h1: {}, h2: {}, h3: {}\nAccuracy: {:.4f}\nAUC: {:.4f}\n\n'.format(h1, h2, h3, accuracy, auc))

Epoch 1/20
12021/12021 [==============================] - 3s 215us/step - loss: 0.3987 - acc: 0.8313 - auc: 0.8425
Epoch 2/20
12021/12021 [==============================] - 1s 82us/step - loss: 0.2741 - acc: 0.8822 - auc: 0.9194
Epoch 3/20
12021/12021 [==============================] - 1s 82us/step - loss: 0.2741 - acc: 0.8860 - auc: 0.9340
Epoch 4/20
12021/12021 [==============================] - 1s 80us/step - loss: 0.2630 - acc: 0.8893 - auc: 0.9392
Epoch 5/20
12021/12021 [==============================] - 1s 80us/step - loss: 0.2501 - acc: 0.8923 - auc: 0.9438
Epoch 6/20
12021/12021 [==============================] - 1s 87us/step - loss: 0.2499 - acc: 0.8952 - auc: 0.9467
Epoch 7/20
12021/12021 [==============================] - 1s 83us/step - loss: 0.2423 - acc: 0.8991 - auc: 0.9490
Epoch 8/20
12021/12021 [==============================] - 1s 80us/step - loss: 0.2391 - acc: 0.8975 - auc: 0.9508
Epoch 9/20
12021/12021 [==============================] - 1s 80us/step - loss: 0.2384 -